# Green Favoritism: Birth Regions, Nightlights, and Industrial Pollution

**Research question:** Do political leaders channel clean economic development to their birth regions while directing pollution-intensive industry elsewhere?

**Hypothesis (Green Favoritism):**
$$\hat\beta^{\text{lights}} > 0 \quad \text{and} \quad \hat\beta^{\text{NO}_2} \leq 0$$

Birth regions see rising nighttime lights (economic activity) but **not** rising NO₂ (industrial pollution) — i.e., a decoupling of growth from pollution in politically favored districts.

**Data:**
- Political leaders: PLAD (Bomprezzi et al., 2025) — ADM2 birth district coding
- Nightlights: Harmonized VIIRS (Li et al., 2020), 2018–2023
- NO₂: TROPOMI/Sentinel-5P via Google Earth Engine, 2018–2023
- Admin boundaries: GADM v4.1 ADM2 (built by `replication.ipynb`)

> **Prerequisite:** Run `replication.ipynb` first to build the ADM2 boundary cache (`data/gadm/gadm41_adm2.gpkg`).

In [12]:
import warnings
import zipfile
import urllib.request
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
from rasterstats import zonal_stats
from linearmodels.panel import PanelOLS
import ee

GEE_PROJECT = "gen-lang-client-0840443535"

ee.Authenticate()   # opens browser — only needed once
ee.Initialize(project=GEE_PROJECT)

warnings.filterwarnings("ignore")

ROOT      = Path("..")  # project root
DATA      = ROOT / "data"
NTL_DIR   = DATA / "nightlights"
PLAD_PATH = DATA / "political leaders" / "PLAD_April_2024.dta"

# Years covered by TROPOMI (and the VIIRS nightlights panel built below)
ANALYSIS_YEARS = range(2018, 2024)

## 1. Load shared boundary data

In [13]:
GADM_DIR   = DATA / "gadm"
ADM2_CACHE = GADM_DIR / "gadm41_adm2.gpkg"

assert ADM2_CACHE.exists(), (
    f"ADM2 boundary cache not found at {ADM2_CACHE}.\n"
    "Run replication.ipynb first to build it."
)

adm2 = gpd.read_file(ADM2_CACHE)
print(f"Loaded {len(adm2):,} ADM2 regions")
adm2.head(2)

Loaded 47,986 ADM2 regions


,GID_0,GID_1,GID_2,NAME_0,NAME_1,NAME_2,geometry
0,ATA,,,Antarctica,,,"MULTIPOLYGON (((-169.00626 -83.61875, -169.004..."
1,UKR,?,?,Ukraine,?,?,"MULTIPOLYGON (((30.59167 50.41236, 30.60611 50..."


## 2. Build PLAD treatment variables (2018–2023)

Same pipeline as `replication.ipynb`, but the spell expansion covers 2018–2023 (TROPOMI era) instead of 1992–2013.

In [14]:
plad = pd.read_stata(PLAD_PATH)
plad = plad[plad["foreign_leader"] == "0"].copy()

# Use gid_2 (ADM2 birth district) if available; fall back to gid_1
BIRTH_GID = "gid_2" if "gid_2" in plad.columns else "gid_1"
plad = plad[plad[BIRTH_GID].str.strip() != "."].copy()
plad["startyear"] = plad["startyear"].astype(int)
plad["endyear"]   = plad["endyear"].astype(int)
plad = plad[plad["archigos_id"].str.strip() != "."].copy()
plad = plad.sort_values(["gid_0", "startyear"]).reset_index(drop=True)

print(f"Using birth region column: {BIRTH_GID}")
print(f"Leaders: {len(plad)}  |  countries: {plad['country'].nunique()}")

Using birth region column: gid_2
Leaders: 820  |  countries: 151


In [15]:
# Fix transition-year overlap (same as replication.ipynb)
plad_fixed = plad.copy().reset_index(drop=True)
for gid_0, group in plad_fixed.groupby("gid_0"):
    idxs = group.index.tolist()
    for i in range(len(idxs) - 1):
        curr, nxt = idxs[i], idxs[i + 1]
        if plad_fixed.loc[curr, "endyear"] >= plad_fixed.loc[nxt, "startyear"]:
            plad_fixed.loc[curr, "endyear"] = plad_fixed.loc[nxt, "startyear"] - 1

# Expand spells into birth-district × year rows for ANALYSIS_YEARS
YEAR_MIN, YEAR_MAX = min(ANALYSIS_YEARS), max(ANALYSIS_YEARS)
rows = []
for idx, row in plad_fixed.iterrows():
    for y in range(max(int(row["startyear"]), YEAR_MIN),
                   min(int(row["endyear"]),   YEAR_MAX) + 1):
        rows.append({"GID_2": row[BIRTH_GID], "GID_0": row["gid_0"],
                     "year": y, "spell_id": idx})

leader_years = pd.DataFrame(rows)
leader_years = leader_years.drop_duplicates(subset=["GID_2", "year"])
leader_years["birth_region_leader"] = 1

spell_map = (leader_years[["GID_0", "year", "spell_id"]]
             .drop_duplicates(subset=["GID_0", "year"]))

print(f"Birth-district × year obs: {len(leader_years)}")
print(f"Unique leader spells:       {spell_map['spell_id'].nunique()}")
print(f"Year range in treatment:    {leader_years['year'].min()}–{leader_years['year'].max()}")

Birth-district × year obs: 382
Unique leader spells:       100
Year range in treatment:    2018–2023


## 3. Nightlights panel (VIIRS, 2018–2023)

The harmonized Li et al. (2020) VIIRS rasters are named `Harmonized_DN_NTL_{year}_simVIIRS.tif`.
Download them from the [dataset page](https://doi.org/10.7910/DVN/YGIVCD) and place them in `data/nightlights/`.

In [16]:
VIIRS_CACHE = DATA / "nightlights_adm2_viirs_panel.parquet"

if VIIRS_CACHE.exists():
    print(f"Loading cached VIIRS panel from {VIIRS_CACHE}")
    ntl_panel = pd.read_parquet(VIIRS_CACHE)
else:
    frames = []
    for year in ANALYSIS_YEARS:
        rpath = NTL_DIR / f"Harmonized_DN_NTL_{year}_simVIIRS.tif"
        if not rpath.exists():
            print(f"  Skipping {year} — raster not found at {rpath}")
            continue
        print(f"  Processing {year}...", end=" ", flush=True)
        stats = zonal_stats(adm2.geometry, str(rpath), stats=["mean"], nodata=0)
        frames.append(pd.DataFrame({
            "GID_2":    adm2["GID_2"].values,
            "GID_1":    adm2["GID_1"].values,
            "GID_0":    adm2["GID_0"].values,
            "year":     year,
            "ntl_mean": [s["mean"] for s in stats],
        }))
        print(f"done")

    assert frames, "No VIIRS rasters found — download them to data/nightlights/ first."
    ntl_panel = pd.concat(frames, ignore_index=True)
    ntl_panel.to_parquet(VIIRS_CACHE)
    print(f"\nSaved VIIRS panel to {VIIRS_CACHE}")

print(f"VIIRS panel: {len(ntl_panel):,} obs | "
      f"{ntl_panel['GID_2'].nunique():,} regions | "
      f"years: {sorted(ntl_panel['year'].unique())}")

  Processing 2018... done
  Processing 2019... done
  Processing 2020... done
  Processing 2021... done
  Processing 2022... done
  Processing 2023... done

Saved VIIRS panel to ../data/nightlights_adm2_viirs_panel.parquet
VIIRS panel: 287,916 obs | 47,986 regions | years: [np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)]


## 4. NO₂ data collection (TROPOMI via Google Earth Engine)

We use **TROPOMI/Sentinel-5P** tropospheric NO₂ (2018–present, ~5.5 km) aggregated to ADM2 level via GEE's `reduceRegions`.

### One-time GEE setup
1. Run the cell below to create `data/gadm/gadm41_adm2.zip` (cleaned shapefile for upload)
2. Go to [code.earthengine.google.com](https://code.earthengine.google.com) → **Assets** → **New** → **Shape files** → upload the zip
3. Set the asset ID to `projects/YOUR_PROJECT/assets/gadm41_adm2`
4. Set `GEE_PROJECT` in the next cell and run the task

In [ ]:
import shutil
from shapely.geometry import box

SHP_DIR = DATA / "gadm" / "gadm41_adm2_shp"
SHP_ZIP = DATA / "gadm" / "gadm41_adm2.zip"

if SHP_ZIP.exists():
    print(f"Shapefile zip already exists: {SHP_ZIP}  ({SHP_ZIP.stat().st_size / 1e6:.1f} MB)")
    print("Delete it and re-run this cell if you need to rebuild.")
else:
    if SHP_DIR.exists():
        shutil.rmtree(SHP_DIR)

    shp_adm2 = adm2[["GID_0", "GID_1", "GID_2", "geometry"]].copy()

    # Fix self-intersecting geometries
    shp_adm2["geometry"] = shp_adm2["geometry"].buffer(0)

    # Clip to valid WGS84 bounds — removes out-of-range vertices that cause GEE errors
    valid_bounds = box(-180, -90, 180, 90)
    shp_adm2["geometry"] = shp_adm2["geometry"].intersection(valid_bounds)

    # Drop empty/null geometries left after clipping
    shp_adm2 = shp_adm2[shp_adm2.geometry.notna() & ~shp_adm2.geometry.is_empty]

    # Simplify to reduce vertex count (GEE limit: 1M vertices per feature)
    shp_adm2["geometry"] = shp_adm2["geometry"].simplify(0.01, preserve_topology=True)

    shp_adm2.to_file(SHP_DIR, driver="ESRI Shapefile")

    with zipfile.ZipFile(SHP_ZIP, "w", zipfile.ZIP_DEFLATED) as zf:
        for f in SHP_DIR.glob("*"):
            zf.write(f, f.name)

    print(f"Features: {len(shp_adm2):,}")
    print(f"Created {SHP_ZIP}  ({SHP_ZIP.stat().st_size / 1e6:.1f} MB)")
    print(f"Upload this zip to GEE via the web UI.")

In [19]:
# ── Update these lines ─────────────────────────────────────────────────────
GEE_PROJECT  = "gen-lang-client-0840443535"                          # ← change this
ASSET_ID     = f"projects/{GEE_PROJECT}/assets/gadm41_adm2"
DRIVE_FOLDER = "GreenFavoritism"
# ───────────────────────────────────────────────────────────────────────────

NO2_CSV         = DATA / "no2_adm2_annual.csv"
NO2_PANEL_CACHE = DATA / "no2_adm2_panel.parquet"

ee.Initialize(project=GEE_PROJECT)
print("GEE initialized")

GEE initialized


In [20]:
# Submit GEE task: annual mean TROPOMI NO₂ per ADM2 district.
# Skips if data is already downloaded.

if NO2_CSV.exists() or NO2_PANEL_CACHE.exists():
    print("NO₂ data already present — skipping GEE task.")
else:
    adm2_fc = ee.FeatureCollection(ASSET_ID)
    s5p = (
        ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_NO2")
        .select("tropospheric_NO2_column_number_density")
    )

    def annual_stats(year):
        year  = ee.Number(year).toInt()
        start = ee.Date.fromYMD(year, 1, 1)
        end   = ee.Date.fromYMD(year.add(1), 1, 1)
        img   = s5p.filterDate(start, end).mean()
        stats = img.reduceRegions(
            collection=adm2_fc,
            reducer=ee.Reducer.mean().setOutputs(["no2"]),
            scale=5500,
            crs="EPSG:4326",
        )
        return stats.map(lambda f: f.set("year", year))

    all_fc = ee.FeatureCollection(
        ee.List(list(ANALYSIS_YEARS)).map(annual_stats)
    ).flatten()

    task = ee.batch.Export.table.toDrive(
        collection=all_fc,
        description="NO2_ADM2_annual_2018_2023",
        folder=DRIVE_FOLDER,
        fileNamePrefix="no2_adm2_annual",
        fileFormat="CSV",
        selectors=["GID_0", "GID_2", "year", "no2"],
    )
    task.start()
    print(f"Task submitted: {task.id}")
    print(f"Monitor: https://code.earthengine.google.com/tasks")
    print(f"When done, download 'no2_adm2_annual.csv' from Google Drive ({DRIVE_FOLDER}/)")
    print(f"to: {DATA}/")

Task submitted: P577VLAFHFYJM63G57LM7LS3
Monitor: https://code.earthengine.google.com/tasks
When done, download 'no2_adm2_annual.csv' from Google Drive (GreenFavoritism/)
to: ../data/


In [ ]:
# Load NO₂ panel from downloaded CSV (run after GEE task completes)

if NO2_PANEL_CACHE.exists():
    print(f"Loading cached NO₂ panel from {NO2_PANEL_CACHE}")
    no2_panel = pd.read_parquet(NO2_PANEL_CACHE)
else:
    assert NO2_CSV.exists(), (
        f"NO₂ CSV not found at {NO2_CSV}.\n"
        f"Complete the GEE task above, then download 'no2_adm2_annual.csv'\n"
        f"from Google Drive ({DRIVE_FOLDER}/) into {DATA}/"
    )
    no2_panel = pd.read_csv(NO2_CSV)
    no2_panel = no2_panel.rename(columns={"no2": "no2_mean"})
    no2_panel = no2_panel.dropna(subset=["no2_mean"])
    no2_panel = no2_panel[no2_panel["no2_mean"] >= 0]   # drop fill values
    no2_panel["no2_mean"] = no2_panel["no2_mean"] * 1e6  # mol/m² → µmol/m²
    no2_panel.to_parquet(NO2_PANEL_CACHE)
    print(f"Saved NO₂ panel to {NO2_PANEL_CACHE}")

print(f"NO₂ panel: {len(no2_panel):,} obs | "
      f"{no2_panel['GID_2'].nunique():,} regions | "
      f"years: {sorted(no2_panel['year'].unique())}")
no2_panel.head()

## 5. Build joint panel and run Green Favoritism regressions

Merge VIIRS nightlights + TROPOMI NO₂ + PLAD treatment for 2018–2023, then run:

$$Y_{it} = \beta \cdot \text{BirthRegion}_{it} + \alpha_i + \gamma_{ct} + \varepsilon_{it}$$

for $Y \in \{\ln(\text{lights}), \ln(\text{NO}_2)\}$ side by side.

In [ ]:
# Merge nightlights + NO₂ + treatment
joint = ntl_panel.merge(no2_panel[["GID_2", "year", "no2_mean"]], on=["GID_2", "year"], how="inner")
joint = joint.merge(leader_years[["GID_2", "year", "birth_region_leader"]], on=["GID_2", "year"], how="left")
joint["birth_region_leader"] = joint["birth_region_leader"].fillna(0).astype(int)
joint = joint.merge(spell_map[["GID_0", "year", "spell_id"]], on=["GID_0", "year"], how="left")
joint["spell_id"] = joint["spell_id"].fillna(
    joint["GID_0"] + "_" + joint["year"].astype(str) + "_noleader"
).astype(str)

joint = joint.sort_values(["GID_2", "year"])
joint["brl_lag1"] = joint.groupby("GID_2")["birth_region_leader"].shift(1).fillna(0).astype(int)

joint["ln_ntl"]       = np.log(joint["ntl_mean"] + 0.01)
joint["ln_no2"]       = np.log(joint["no2_mean"] + 0.01)
joint["country_year"] = joint["GID_0"] + "_" + joint["year"].astype(str)
joint = joint.dropna(subset=["ntl_mean", "no2_mean"])

print(f"Joint panel: {len(joint):,} obs | {joint['GID_2'].nunique():,} regions | "
      f"years: {sorted(joint['year'].unique())}")
print(f"Treated obs (birth_region_leader=1): {joint['birth_region_leader'].sum()}")

In [ ]:
# Green Favoritism regressions: lights vs NO₂, Lag 0 and Lag 1
# FE: ADM2 region + country×year  |  Clustering: leader-period

joint_idx = joint.set_index(["GID_2", "year"])

specs    = [("Lag 0", "birth_region_leader"), ("Lag 1", "brl_lag1")]
outcomes = [("ln_ntl", "log(Nightlights)"), ("ln_no2", "log(NO₂)")]

rows = []
for lag_label, var in specs:
    for outcome_col, outcome_label in outcomes:
        model = PanelOLS.from_formula(
            f"{outcome_col} ~ {var} + EntityEffects",
            data=joint_idx,
            other_effects=joint_idx["country_year"],
            drop_absorbed=True,
        )
        res   = model.fit(cov_type="clustered", clusters=joint_idx["spell_id"])
        coef  = res.params[var]
        se    = res.std_errors[var]
        pval  = res.pvalues[var]
        stars = "***" if pval < 0.01 else "**" if pval < 0.05 else "*" if pval < 0.1 else ""
        rows.append({
            "Spec":    lag_label,
            "Outcome": outcome_label,
            "Coeff":   f"{coef:.4f}{stars}",
            "SE":      f"({se:.4f})",
            "p":       f"{pval:.3f}",
            "N":       f"{res.nobs:,}",
        })

results_df = pd.DataFrame(rows).set_index(["Spec", "Outcome"])
print("Green Favoritism test")
print("FE: Region (ADM2) + Country×Year  |  Clustering: Leader-period\n")
print(results_df.to_string())
print("\nSignificance: *** p<0.01, ** p<0.05, * p<0.1")
print("\nGreen Favoritism = lights coeff > 0  AND  NO₂ coeff ≤ 0")